In [1]:
import pandas as pd
import os
import numpy as np

In [ ]:
import os
print(os.getcwd())

In [4]:
os.chdir("../")

In [ ]:
import os
print(os.getcwd())

In [81]:
df = pd.read_csv('artifacts\data_ingestion\Churn_Modelling.csv')

In [82]:
df = df.dropna()

In [ ]:
df.isnull().sum()

In [84]:
df = df.drop_duplicates()

In [85]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.info()

In [ ]:
df.describe()

In [88]:
unique_values_df = pd.DataFrame({
    'Column': df.columns,
    'Unique Values': [df[col].unique() for col in df.columns],
    'No. of Unique Values': [df[col].nunique() for col in df.columns]
})

In [ ]:
unique_values_df

In [90]:
df['IsActiveMember'] = df['IsActiveMember'].astype(int)
df['HasCrCard'] = df['HasCrCard'].astype(int)
df['Age'] = df['Age'].astype(int)

In [91]:
df = df.drop(['RowNumber','CustomerId', 'Surname'], axis=1)

In [92]:
mapping = {'Male': 0, 'Female': 1}
df['Gender'] = df['Gender'].map(mapping)

In [ ]:
df.info()

In [94]:
X = df.drop('Exited', axis=1)
y = df['Exited']

In [95]:
num_cols = X.select_dtypes(include=np.number).columns.to_list()
cat_cols = X.select_dtypes(exclude=np.number).columns.to_list()

In [ ]:
print(num_cols)
print(cat_cols)

In [97]:
from sklearn.model_selection import train_test_split

In [98]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [99]:
X_train.shape

(7996, 10)

In [ ]:
type(X_train), type(y_train)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [103]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [104]:
num_pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('one_hot_enc', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))
])

transformer = ColumnTransformer(transformers=[
    ('num_pipeline', num_pipeline, num_cols),
    ('cat_pipeline', cat_pipeline, cat_cols),
    ],remainder='passthrough',
    n_jobs=-1
)

In [105]:
X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.transform(X_test)

In [ ]:
X_train_transformed.shape

In [ ]:
type(X_train_transformed)

In [ ]:
X_train_transformed[0]

In [ ]:
feature_names = transformer.get_feature_names_out()
feature_names

In [110]:
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=feature_names)
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=feature_names)

In [ ]:
X_train_transformed_df

In [112]:
y_train_df = y_train.to_frame().reset_index(drop=True)
y_test_df = y_test.to_frame().reset_index(drop=True)

In [113]:
train_preprocessed = pd.concat([X_train_transformed_df, y_train_df], axis=1)
test_preprocessed = pd.concat([X_test_transformed_df, y_test_df], axis=1)

In [ ]:
train_preprocessed